In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 

input_shape = (224,224,3)
classes_num = 1000
bn_axis = 3

def conv2d_bn(x, filters, kernel_size,
              padding='same', stride=1, activation='relu',
              name=None):
  x = layers.Conv2D(filters, kernel_size, stride, padding)(x)
  x = layers.BatchNormalization()(x)
  return layers.Activation(activation, name=name)(x)

def indentity_res_block(input, filters, stage=None, block=None):
  filter_1, filter_2, filter_3 = filters
  
  name_base = 'conv_' + str(stage) + '_id_block' + str(block) #이름 짓는것, 중요치 않음

  x = conv2d_bn(input, filter_1, 1, name=name_base+'_conv1')  
  x = conv2d_bn(x, filter_2, 3, name=name_base+'_conv2')
  x = conv2d_bn(x, filter_3, 1, name=name_base+'_conv3')

  x = layers.Add(name=name_base+'_add')([x , input])
  
  return layers.Activation('relu', name=name_base+'_relu')(x)

def conv_res_block(input, filters, stage=None, block=None, stride=2):
  filter_1, filter_2, filter_3 = filters

  name_base = 'conv_' + str(stage) + '_conv_block' + str(block)

  x = conv2d_bn(input, filter_1, 1, stride=stride, name=name_base+'_conv1')  
  x = conv2d_bn(x, filter_2, 3, name=name_base+'_conv2')
  x = conv2d_bn(x, filter_3, 1, name=name_base+'_conv3')

  # Conv Shortcut
  shortcut = conv2d_bn(input, filter_3, 1, stride=stride, activation=None,
                       name=name_base+'_short_conv')
  
  x = layers.Add(name=name_base+'_ad')([x , shortcut])

  return layers.Activation('relu', name=name_base+'_relu')(x)

![1.PNG](...)

input = keras.Input(input_shape)

x = conv2d_bn(input, 64, 7, 'same',2, name='stem_conv')
x = layers.MaxPooling2D(3, 2, 'same', name='stem_pool')(x)

x = conv_res_block(x, [64, 64, 256], stage=1, block=1, stride=1)
x = indentity_res_block(x, [64, 64, 256], stage=1, block=2)
x = indentity_res_block(x, [64, 64, 256], stage=1, block=3)

x = conv_res_block(x, [128, 128, 512], stage=2, block=1, stride=2)
x = indentity_res_block(x, [128, 128, 512], stage=2, block=2)
x = indentity_res_block(x, [128, 128, 512], stage=2, block=3)
x = indentity_res_block(x, [128, 128, 512], stage=2, block=4)

x = conv_res_block(x, [256, 256, 1024], stage=3, block=1, stride=2)
x = indentity_res_block(x, [256, 256, 1024], stage=3, block=2)
x = indentity_res_block(x, [256, 256, 1024], stage=3, block=3)
x = indentity_res_block(x, [256, 256, 1024], stage=3, block=4)
x = indentity_res_block(x, [256, 256, 1024], stage=3, block=5)
x = indentity_res_block(x, [256, 256, 1024], stage=3, block=6)

x = conv_res_block(x, [512, 512, 2048], stage=4, block=1, stride=2)
x = indentity_res_block(x, [512, 512, 2048], stage=4, block=2)
x = indentity_res_block(x, [512, 512, 2048], stage=4, block=3)

x = layers.GlobalAveragePooling2D(name = 'avg_pool')(x)
output = layers.Dense(classes_num, 'softmax', name = 'output')(x)

model = keras.Model(
    inputs=input,
    outputs=output
)

model.summary()

keras.utils.plot_model(model, show_shapes = True)